<a href="https://colab.research.google.com/github/Ahella-Bassem-Mohammed/Stroke_Pre-Detection_Analysis/blob/main/Stroke_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install category_encoders

In [2]:
# import necessary libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
# Enable iterative imputer before importing it
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from category_encoders.target_encoder import TargetEncoder

import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv('/content/full-stroke-data.csv')
df.head(5)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [4]:
df['gender'].value_counts()

,count
gender,
Female,2994
Male,2115
Other,1


In [5]:
df[df["gender"]=="Other"]

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
3116,56156,Other,26.0,0,0,No,Private,Rural,143.33,22.4,formerly smoked,0


In [6]:
df= df[df['gender'].isin(['Male', 'Female'])]

In [7]:
df.iloc[3114:3118,:]

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
3114,67032,Male,42.0,0,0,No,Govt_job,Urban,115.21,28.7,Unknown,0
3115,39784,Female,72.0,0,0,Yes,Self-employed,Urban,65.12,28.3,never smoked,0
3117,15230,Female,9.0,0,0,No,children,Rural,80.55,15.1,Unknown,0
3118,25218,Female,31.0,0,0,Yes,Govt_job,Urban,88.20,22.7,never smoked,0


In [8]:
index=[i for i in range(df.shape[0])]
df.index=index
df.iloc[3114:3118,:]

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
3114,67032,Male,42.0,0,0,No,Govt_job,Urban,115.21,28.7,Unknown,0
3115,39784,Female,72.0,0,0,Yes,Self-employed,Urban,65.12,28.3,never smoked,0
3116,15230,Female,9.0,0,0,No,children,Rural,80.55,15.1,Unknown,0
3117,25218,Female,31.0,0,0,Yes,Govt_job,Urban,88.20,22.7,never smoked,0


In [9]:
df['gender'].value_counts()

,count
gender,
Female,2994
Male,2115


In [10]:
df[["ever_married","Residence_type","gender"]]=pd.get_dummies(df[["ever_married","Residence_type","gender"]],drop_first=True)
df[["ever_married", "Residence_type", "gender"]] = df[["ever_married", "Residence_type", "gender"]].astype(int)
df.head(5)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,1,67.0,0,1,1,Private,1,228.69,36.6,formerly smoked,1
1,51676,0,61.0,0,0,1,Self-employed,0,202.21,NaN,never smoked,1
2,31112,1,80.0,0,1,1,Private,0,105.92,32.5,never smoked,1
3,60182,0,49.0,0,0,1,Private,1,171.23,34.4,smokes,1
4,1665,0,79.0,1,0,1,Self-employed,0,174.12,24.0,never smoked,1


In [11]:
enc=TargetEncoder()
to_encode="work_type"
enc.fit(X=df[to_encode],y=df["stroke"])
encoded = enc.transform(df[to_encode])
df["work_type"] = encoded["work_type"]

In [12]:
missing_values = df.isnull().sum()

print("Missing values in each column:")
print(missing_values)


Missing values in each column:
id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64


In [13]:
df['work_type'].value_counts()

,count
work_type,
0.050958,2924
0.079365,819
0.002911,687
0.050228,657
0.021940,22


In [14]:
children_rows = df[df['work_type'] == 'children']

children_above_18 = children_rows[children_rows['age'] >= 18]

print(f"Rows where work_type is 'children' and age is 18 or above:")
print(children_above_18)

Rows where work_type is 'children' and age is 18 or above:
Empty DataFrame
Columns: [id, gender, age, hypertension, heart_disease, ever_married, work_type, Residence_type, avg_glucose_level, bmi, smoking_status, stroke]
Index: []


In [15]:
df['work_type'] = df['work_type'].map({
    'Private': 0,
    'Self-employed': 1,
    'Govt_job': 2,
    'children': 3,
    'Never_worked': 4
})

In [16]:
df['smoking_status'].value_counts()

,count
smoking_status,
never smoked,1892
Unknown,1544
formerly smoked,884
smokes,789


In [17]:
unknown_percentage = (df['smoking_status'].value_counts()['Unknown'] / len(df)) * 100
print(f"Percentage of 'Unknown' in smoking_status: {unknown_percentage:.2f}%")

Percentage of 'Unknown' in smoking_status: 30.22%


In [18]:
missing_percentage = df['bmi'].isnull().sum() / len(df) * 100
print(f"Percentage of missing values in 'bmi': {missing_percentage:.2f}%")

Percentage of missing values in 'bmi': 3.93%


In [19]:
smoking_mapper={"never smoked":0,"formerly smoked":1,"smokes":2,"Unknown":np.nan}
for i in range(df.shape[0]):
    status=df.loc[i,"smoking_status"] # Use .loc for safer indexing
    if pd.notna(status): # Check if 'status' is not NaN before mapping
        df.loc[i,"smoking_status"]=smoking_mapper.get(status,status) # Use get to handle missing keys, keep orginal value
# or df.loc[i,"smoking_status"]=smoking_mapper[status]
df["smoking_status"].unique()

array([1, 0, 2, nan], dtype=object)

In [20]:
estimator=RandomForestRegressor(max_depth=8)
mice = IterativeImputer(estimator=estimator,random_state=11,skip_complete=True)

In [21]:
impdata=mice.fit_transform(df)

In [22]:
impdata=pd.DataFrame(impdata,columns=df.columns)

ValueError: Shape of passed values is (5109, 11), indices imply (5109, 12)

In [ ]:
impdata.isnull().sum()

In [ ]:
impdata.head(10)